In [1]:
from fuzzywuzzy import fuzz
import pandas as pd 
import numpy as np

In [2]:
ent_df = pd.read_csv('entity.csv')
ent_df.head()

,DB_ID,Name,Blck_List
0,14173,ALVAREZ CARLOS,0
1,14174,TARANTOLA SALVATORE,0
2,14175,HOME DEVCO TIVOLI ISLES LLC,1
3,14176,LEVIN MICHAEL,0
4,14177,ROSS NICHOLAS J,0


This code was used to find most commont word tokens.  
I used this to build an ignore list where I am filtering out common  
words to describe a business. IE  Corp, Inc, Co

```python
from collections import Counter
ent_counter = Counter()
for names in ent_df['Name'].str.split():
    ent_counter.update(names)
with open('common.csv', mode='w') as f:
    print("Name, count", file=f)
    for c in  ent_counter.most_common(100):
        print(f'{c[0]},{c[1]}', file=f)
```

In [3]:
ignore = pd.read_csv('common.csv')
ignore_list = ignore['Name'].to_list()

In [4]:
def clean(e):
    return all([t not in ignore_list for t in e.split()])

ent_df_clean = ent_df[ent_df['Name'].map(clean)].copy()
# creating a new pf object not ref

In [5]:
print(f'counts: original -> {len(ent_df)} cleaned -> {len(ent_df_clean)}')

counts: original -> 17187 cleaned -> 13865


In [6]:
ent_df_clean['token_count'] = ent_df_clean['Name'].str.split().str.len()
ent_df_clean.head(10)

,DB_ID,Name,Blck_List,token_count
0,14173,ALVAREZ CARLOS,0,2
1,14174,TARANTOLA SALVATORE,0,2
3,14176,LEVIN MICHAEL,0,2
4,14177,ROSS NICHOLAS J,0,3
5,14178,PISCATELLI ALAN,0,2
6,14179,SCHNEIDER ROBERT,0,2
7,14180,FOBBS RAMONA A,0,3
8,14181,"KELLER FRED ESTATE,SHEPHERD DANIEL J PERSONAL ...",1,7
9,14182,"SHEPHERD DANIEL J PERSONAL REPRESENTATIVE,KELL...",1,7
10,14183,MICHEL JEANTRAL,0,2


In [7]:
# find same tokens simple mispelling
def join_first(tokens):
    return ''.join([s[0] for s in tokens])
ent_df_clean['grp_1'] = ent_df_clean.Name.str.split().apply(join_first)
#ent_df_clean = ent_df_clean.drop('grp_2', axis=1)

In [8]:
ent_df_clean.size

69325

In [92]:
grps_simple = ent_df_clean.groupby('grp_1')
grp_1 = grps_simple.filter(lambda x: x['grp_1'].count() > 1)['grp_1'].unique()

In [93]:
grp_test = ['AA',]

In [179]:
ent_df_clean['alias_match'], ent_df_clean['score'] = np.nan, np.nan

In [180]:
for count, group_name in enumerate(grp_1):
    this_group = grps_simple.get_group(group_name)
    this_group = this_group.sort_values('Name')
    progress = len(this_group)*.125//1
    conunt = 1
    for lidx in this_group.index:
        if pd.isna(ent_df_clean['alias_match'].loc[lidx]):
            ent_df_clean.loc[lidx, 'alias_match'] = this_group['Name'].loc[lidx]
            ent_df_clean.loc[lidx, 'score'] = 110
            for ridx in this_group.index:
                if lidx == ridx: 
                    continue
                if pd.isna(ent_df_clean['alias_match'].loc[ridx]):
                    score = fuzz.partial_ratio(this_group['Name'].loc[lidx], this_group['Name'].loc[ridx])
                    if score > 93:
                        lname = this_group.loc[lidx, 'Name']
                        rname = this_group.loc[ridx, 'Name']
                        #print(f'Matched {lname} with {rname}')
                        ent_df_clean.loc[ridx, 'alias_match'] = this_group['Name'].loc[lidx]
                        ent_df_clean.loc[ridx, 'score'] = score
    if count+1>100 and (count+1)%100 == 0:
            print(f"progress: {100*(count+1)/len(grp_1)}% {count+1} groups")
print(f"complete: 100% {count+1} groups")

progress: 15.797788309636651% 200 groups
progress: 23.696682464454977% 300 groups
progress: 31.595576619273302% 400 groups
progress: 39.494470774091624% 500 groups
progress: 47.39336492890995% 600 groups
progress: 55.292259083728275% 700 groups
progress: 63.191153238546605% 800 groups
progress: 71.09004739336493% 900 groups
progress: 78.98894154818325% 1000 groups
progress: 86.88783570300158% 1100 groups
progress: 94.7867298578199% 1200 groups
complete: 100% 1266 groups


In [183]:
ent_df_clean[ent_df_clean['score'] < 110]

,DB_ID,Name,Blck_List,token_count,grp_1,alias_match,score
10,14183,MICHEL JEANTRAL,0,2,MJ,MICHEL JEAN,100.0
87,14260,RODRIGUEZ ANGELO,0,2,RA,RODRIGUEZ ANGEL,100.0
140,14313,RODRIGUEZ ALEXANDER,0,2,RA,RODRIGUEZ ALEX,100.0
141,14314,MILLER RICHARD,0,2,MR,MILLER R,100.0
197,14370,GARCIA ALEXANDER,0,2,GA,GARCIA ALEX,100.0
198,14371,WILLIAMS JOHNNY,0,2,WJ,WILLIAMS JOHN,100.0
408,14581,SMITH DANIELLE,0,2,SD,SMITH DANIEL,100.0
501,14674,HERNANDEZ JOSE ANTONIO,0,3,HJA,HERNANDEZ JOSE A,100.0
514,14687,ALLEN STEVEN,0,2,AS,ALLEN STEVE,100.0
571,14744,PEREZ JULIAN,0,2,PJ,PEREZ JULIA,100.0


In [ ]:
ent_df_clean[(ent_df_clean['token_count'] == 2)].count()

In [61]:
who.match('Zuniga Alberto', 'Zuniga Albert')

False

In [85]:
40*100 * .125

500.0

In [150]:
2 % (1266 * .125//1)

2.0

In [129]:
1266 % (158.25*2) == 0

True

In [156]:
2*.125//1

0.0

In [172]:
100%200

100